In [1]:
from datasets import load_dataset
import pandas as pd


/Users/krithis/Documents/nlp_topic_modeling/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

#https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

data = load_dataset('McAuley-Lab/Amazon-Reviews-2023','raw_review_Gift_Cards',trust_remote_code=True)
#meta=load_dataset('McAuley-Lab/Amazon-Reviews-2023','raw_meta_Gift_Cards',trust_remote_code=True)

In [3]:
#sample what one row of the data looks like
print(data["full"][0])

{'rating': 5.0, 'title': 'Great gift', 'text': 'Having Amazon money is always good.', 'images': [], 'asin': 'B00IX1I3G6', 'parent_asin': 'B00IX1I3G6', 'user_id': 'AHZ6XMOLEWA67S3TX7IWEXXGWSOA', 'timestamp': 1549866158332, 'helpful_vote': 0, 'verified_purchase': True}


In [4]:
dataset=data["full"]
print(dataset)


Dataset({
    features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
    num_rows: 152410
})


In [5]:

# Create an empty list to store rows
rows = []
for review in dataset:
    rating=review['rating']
    title=review['title']
    text=review['text']
    images=review['images']
    asin=review['asin']
    parent_asin=review['parent_asin']
    user_id=review['user_id']
    timestamp=review['timestamp']
    helpful_vote=review['helpful_vote']
    verified_purchase=review['verified_purchase']
    rows.append([rating, title, text, images, asin, parent_asin, user_id, timestamp, helpful_vote, verified_purchase])

reviews = pd.DataFrame(rows, columns=['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'])


In [6]:
reviews.head()


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Great gift,Having Amazon money is always good.,[],B00IX1I3G6,B00IX1I3G6,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,1549866158332,0,True
1,5.0,amazon gift card,Always the perfect gift. I have never given o...,[],B005ESMMWW,B005ESMMWW,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1599875158120,0,False
2,5.0,perfect gift,When you have a person who is hard to shop for...,[],B01K8RIM5Y,B005S28ZES,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1535939929239,27,True
3,5.0,Nice looking,The tin is a nice touch and pretty large. It'...,[],B0091JKVU0,B00ADR2LV6,AFZUK3MTBIBEDQOPAK3OATUOUKLA,1418439577000,0,False
4,1.0,Not $10 Gift Cards,I bought this pack of Starbucks Gift cards in ...,[],B00FTGTM5E,B00FTGTIOE,AH5L7ILVA6HYLZOUZIQAWNHVVK3A,1638068808115,2,True


In [7]:
#there is no 0.5 scale
reviews['rating'].value_counts()


rating
5.0    128248
1.0     12326
4.0      6692
3.0      3271
2.0      1873
Name: count, dtype: int64

In [8]:
#liklihood to recommend or NPS is produced on a scale of 1 - 10 , may not be relavent, NPS= (Promoters - Detractors)
#instead, we can take a look at, in eah star category what is the customer talking about?
#start with data preprocessing

#check how many product categories are there - 1137
reviews['parent_asin'].value_counts()


parent_asin
B00IX1I3G6    36863
B00ADR2LV6    14912
B077N4CNVJ     6940
B086KKT3RX     6635
B018F4M89S     6329
              ...  
B002YEWXZU        1
B00ONCTANE        1
B06XW61V3B        1
B002SC04U4        1
B00O988YK2        1
Name: count, Length: 1137, dtype: int64

In [9]:
#start with one category 
df_cat1=reviews[(reviews['parent_asin']=='B00IX1I3G6')]

#drop columns not required for this analysis
df_cat1.drop(['images'],inplace=True,axis=1)

/var/folders/8w/mv0nvzh126z_rwpbhvx_k4mh0000gn/T/ipykernel_59327/617545395.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat1.drop(['images'],inplace=True,axis=1)


In [10]:
df_cat1.head()
df_cat1['asin'].value_counts()

asin
B00IX1I3G6    36863
Name: count, dtype: int64

In [11]:
df_cat1['parent_asin'].value_counts()

parent_asin
B00IX1I3G6    36863
Name: count, dtype: int64

In [12]:
#asin=parent asin
df_cat1.drop(['asin'],inplace=True,axis=1)
df_cat1.head()

/var/folders/8w/mv0nvzh126z_rwpbhvx_k4mh0000gn/T/ipykernel_59327/3461983026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat1.drop(['asin'],inplace=True,axis=1)


,rating,title,text,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Great gift,Having Amazon money is always good.,B00IX1I3G6,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,1549866158332,0,True
6,5.0,Easy gift,Great and easy gift,B00IX1I3G6,AFSCQMP2EPYLGJN7OBTMEXCBE2OQ,1602104097716,0,True
14,5.0,Easy..,Reloading is easy. Shopping is easy. Amazon is...,B00IX1I3G6,AFO3G62P2JXCNMWZTAIB56KPG56A,1466368648000,0,True
31,5.0,So easy,Easy to do,B00IX1I3G6,AEINWALDRYT4NTF3M5NC2WZ5YQNA,1587064572520,0,True
32,2.0,Mistake,Hit button by mistake n deduction from credit ...,B00IX1I3G6,AHVPSZCUQH3UMQCGQSDZ7BZVUHNQ,1576194477554,0,True


In [13]:
df_cat1.shape #36.8K reviews 

#check if there are any reviews with empty text

(36863, 8)

In [14]:
df_cat1['text'].isna().any()


False

In [15]:
df_cat1['text'].isnull().any()

False

In [16]:
#get an understanding of some general themes in the text
#pre-proessing - text cleaning, tokenization,lowercasing,stopword removal  
#if words are very specific to your business, you can build a custom dictionary of stop words
import nltk
import ssl
#bash '/Applications/Python 3.12/Install Certificates.command'

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('all')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')


In [17]:
# Preprocessing

# Define the path to your stop words file
stop_words_file = 'english_stopwords'
# Read the stop words from the file
with open(stop_words_file, 'r') as file:
    stop_words = file.readlines()
# Strip newline characters and any leading/trailing whitespace from each stop word
stop_words = [word.strip() for word in stop_words]

#list of stop_words
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [18]:
#Reducing words to their roots or stems is known as lemmatization
lemmatizer = WordNetLemmatizer()

#split sentences into words = tokenization, and words into simplest form = lemmatize
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

df_cat1['processed_text'] = df_cat1['text'].apply(preprocess_text)


/var/folders/8w/mv0nvzh126z_rwpbhvx_k4mh0000gn/T/ipykernel_59327/2653384939.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat1['processed_text'] = df_cat1['text'].apply(preprocess_text)


In [19]:
df_cat1.head()

,rating,title,text,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,processed_text
0,5.0,Great gift,Having Amazon money is always good.,B00IX1I3G6,AHZ6XMOLEWA67S3TX7IWEXXGWSOA,1549866158332,0,True,"[amazon, money, always, good]"
6,5.0,Easy gift,Great and easy gift,B00IX1I3G6,AFSCQMP2EPYLGJN7OBTMEXCBE2OQ,1602104097716,0,True,"[great, easy, gift]"
14,5.0,Easy..,Reloading is easy. Shopping is easy. Amazon is...,B00IX1I3G6,AFO3G62P2JXCNMWZTAIB56KPG56A,1466368648000,0,True,"[reloading, easy, shopping, easy, amazon, easy]"
31,5.0,So easy,Easy to do,B00IX1I3G6,AEINWALDRYT4NTF3M5NC2WZ5YQNA,1587064572520,0,True,[easy]
32,2.0,Mistake,Hit button by mistake n deduction from credit ...,B00IX1I3G6,AHVPSZCUQH3UMQCGQSDZ7BZVUHNQ,1576194477554,0,True,"[hit, button, mistake, n, deduction, credit, c..."


In [20]:
df_cat1['rating'].value_counts()

rating
5.0    31217
1.0     2365
4.0     1909
3.0      919
2.0      453
Name: count, dtype: int64

In [21]:
reviews5=df_cat1['processed_text'][(df_cat1['rating'])==5]
reviews5.head()

0                       [amazon, money, always, good]
6                                 [great, easy, gift]
14    [reloading, easy, shopping, easy, amazon, easy]
31                                             [easy]
38                [really, ready, reload, gift, card]
Name: processed_text, dtype: object

In [22]:
reviews5.shape

(31217,)

In [23]:
type(reviews5)

pandas.core.series.Series

In [29]:
#each review considered as one document. tried converting to one large body of text,  list of words, doesnt work with doc2bow
reviews5=reviews5.tolist()
reviews5

[['amazon', 'money', 'always', 'good'],
 ['great', 'easy', 'gift'],
 ['reloading', 'easy', 'shopping', 'easy', 'amazon', 'easy'],
 ['easy'],
 ['really', 'ready', 'reload', 'gift', 'card'],
 ['really', 'easy', 'load'],
 ['simple', 'easy'],
 ['simple',
  'gift',
  'friend',
  'family',
  'wa',
  'easy',
  'setup',
  'took',
  'ok',
  'second',
  'also',
  'redeem',
  'visa',
  'gift',
  'giving',
  'gift',
  'card',
  'like',
  'isnt',
  'fee',
  'want',
  'amazon',
  'make',
  'cooler',
  'card',
  'animation',
  'receiver',
  'get'],
 ['bonus',
  'rec',
  'promo',
  'amazon',
  'reloading',
  'gift',
  'card',
  'br',
  'br',
  'gift',
  'card',
  'wa',
  'posted',
  'account',
  'quickly'],
 ['nice', 'way', 'save', 'buck', 'overtime', 'reloading', 'gift', 'card'],
 ['know',
  'kid',
  'want',
  'nothing',
  'actually',
  'want',
  'everything',
  'nothing',
  'good',
  'enough',
  'yes',
  'yes',
  'secret',
  'weapon',
  'amazon',
  'gift',
  'card',
  'pulled',
  'showed',
  'love',

In [30]:
type(reviews5)

list

In [31]:
reviews4=df_cat1['processed_text'][(df_cat1['rating'])==4]
reviews3=df_cat1['processed_text'][(df_cat1['rating'])==3]
reviews2=df_cat1['processed_text'][(df_cat1['rating'])==2]
reviews1=df_cat1['processed_text'][(df_cat1['rating'])==1]


In [32]:
reviews4=reviews4.tolist()
reviews3=reviews3.tolist()
reviews2=reviews2.tolist()
reviews1=reviews1.tolist()

In [144]:
# combined_text4 = ' '.join([word for sublist in reviews4 for word in sublist])
# combined_text3 = ' '.join([word for sublist in reviews3 for word in sublist])
# combined_text2 = ' '.join([word for sublist in reviews2 for word in sublist])
# combined_text1 = ' '.join([word for sublist in reviews1 for word in sublist])

In [53]:
from gensim import corpora, models
from gensim.models import LdaModel, LdaMulticore
from gensim.models.coherencemodel import CoherenceModel


In [35]:

#The scipy.linalg functions tri, triu & tril are deprecated and will be removed in SciPy 1.13. pip install scipy==1.13.0
#https://www.analyticsvidhya.com/blog/2022/02/topic-identification-with-gensim-library-using-python/
#create a bag of words of all ratinngs by rating, to see what the themes for each star rating are
#create a dictionary of all terms, key is an ID for each word
bow5 = corpora.Dictionary(reviews5)


In [36]:
bow4 = corpora.Dictionary(reviews4)
bow3 = corpora.Dictionary(reviews3)
bow2 = corpora.Dictionary(reviews2)
bow1 = corpora.Dictionary(reviews1)

In [37]:
#check dictionary, first 10 items
count = 0
for k, v in bow5.iteritems():
    print(k,v)
    count += 1
    if count > 10:
        break

0 always
1 amazon
2 good
3 money
4 easy
5 gift
6 great
7 reloading
8 shopping
9 card
10 ready


In [38]:
count = 0
for k, v in bow5.iteritems():
    count += 1
print(count)

6984


In [65]:
# texts = [['human', 'interface', 'computer']]
# dct = corpora.Dictionary(texts)  # initialize a Dictionary
# dct.add_documents([["cat", "dog", "dog"], ["dog"]])  # add more document (extend the vocabulary)
# dct.doc2bow(["dog", "computer", "non_existent_word"])
# count = 0
# for k, v in dct.iteritems():
#     print(k,v)


In [39]:
#filter extremes: exclude words that are in less than 5 docs
bow5.filter_extremes(no_below=5,no_above=0.9,keep_n= 1000)
count = 0
for k, v in bow5.iteritems():
    #print(k,v)
    count += 1
print(count)
#reduced from 6984 words to 1000
#noticed some non-english/spanish words as well in this list

1000


In [40]:
bow4.filter_extremes(no_below=5, no_above=0.9,keep_n= 1000)
bow3.filter_extremes(no_below=5, no_above=0.9,keep_n= 1000)
bow2.filter_extremes(no_below=5, no_above=0.9,keep_n= 1000)
bow1.filter_extremes(no_below=5, no_above=0.9,keep_n= 1000)

In [41]:
bow_corpus5 = [bow5.doc2bow(doc) for doc in reviews5]

In [42]:
reviews5[5]

['really', 'easy', 'load']

In [50]:
print(bow5[4], bow5[11], bow5[13])

easy really load


In [43]:
document_num = 5
bow_doc_x = bow_corpus5[document_num] #contains id of the word + frequency mappings
bow_doc_x

[(4, 1), (11, 1), (13, 1)]

In [51]:
bow_corpus4 = [bow4.doc2bow(doc) for doc in reviews4]
bow_corpus3 = [bow3.doc2bow(doc) for doc in reviews3]
bow_corpus2 = [bow2.doc2bow(doc) for doc in reviews2]
bow_corpus1 = [bow1.doc2bow(doc) for doc in reviews1]

In [ ]:
#run LDA mode, train on bag of words
#parameters to adjust: number of themes, id2word mappings, #processorts to use for parallelization, 
#alpha (subject distribution per document, Low alpha: Each document comprises a few subjects.) 
#and eta (per-topic word distribution is referred to as Eta, High eta: Each topic contains a variety of terms (topics appear similar to each other),Low eta: Each topic comprises a little number of words.)

In [56]:
lda_model5= LdaMulticore(bow_corpus5, 
 num_topics = 5, 
 id2word = bow5, 
 passes = 10,
 workers = 2)

In [57]:
for idx, topic in lda_model5.print_topics(-1):
    print("Topic: {} nWords: {}".format(idx, topic ))
    print("n")

Topic: 0 nWords: 0.057*"wa" + 0.046*"got" + 0.043*"excellent" + 0.035*"bonus" + 0.028*"problem" + 0.026*"thanks" + 0.024*"extra" + 0.021*"excelente" + 0.018*"free" + 0.017*"reload"
n
Topic: 1 nWords: 0.036*"amazon" + 0.035*"money" + 0.034*"card" + 0.031*"purchase" + 0.028*"like" + 0.020*"account" + 0.018*"time" + 0.017*"credit" + 0.017*"get" + 0.017*"gift"
n
Topic: 2 nWords: 0.131*"card" + 0.125*"gift" + 0.046*"amazon" + 0.044*"balance" + 0.024*"reload" + 0.024*"nice" + 0.023*"wa" + 0.021*"br" + 0.016*"use" + 0.012*"way"
n
Topic: 3 nWords: 0.374*"easy" + 0.137*"use" + 0.068*"fast" + 0.055*"quick" + 0.054*"reload" + 0.049*"convenient" + 0.046*"wa" + 0.030*"simple" + 0.021*"super" + 0.020*"process"
n
Topic: 4 nWords: 0.206*"great" + 0.169*"good" + 0.090*"love" + 0.047*"work" + 0.034*"worked" + 0.032*"way" + 0.030*"money" + 0.025*"product" + 0.023*"deal" + 0.021*"amazon"
n


In [58]:
lda_model4 = LdaMulticore(bow_corpus4, 
 num_topics = 5, 
 id2word = bow4, 
 passes = 10,
 workers = 2)

lda_model3 = LdaMulticore(bow_corpus3, 
 num_topics = 5, 
 id2word = bow3, 
 passes = 10,
 workers = 2)

lda_model2 = LdaMulticore(bow_corpus2, 
 num_topics = 5, 
 id2word = bow2, 
 passes = 10,
 workers = 2)

lda_model1 = LdaMulticore(bow_corpus1, 
 num_topics = 5, 
 id2word = bow1, 
 passes = 10,
 workers = 2)

In [59]:
for idx, topic in lda_model4.print_topics(-1):
    print("Topic: {} nWords: {}".format(idx, topic ))
    print("n")


Topic: 0 nWords: 0.098*"good" + 0.038*"wa" + 0.028*"reload" + 0.024*"nice" + 0.024*"use" + 0.024*"get" + 0.021*"amazon" + 0.020*"easy" + 0.020*"order" + 0.018*"br"
n
Topic: 1 nWords: 0.105*"card" + 0.081*"gift" + 0.050*"amazon" + 0.029*"balance" + 0.024*"purchase" + 0.023*"br" + 0.020*"credit" + 0.017*"use" + 0.017*"money" + 0.014*"way"
n
Topic: 2 nWords: 0.064*"card" + 0.055*"wa" + 0.042*"gift" + 0.024*"reload" + 0.022*"balance" + 0.017*"like" + 0.014*"load" + 0.014*"one" + 0.013*"use" + 0.013*"time"
n
Topic: 3 nWords: 0.160*"easy" + 0.073*"use" + 0.053*"wa" + 0.047*"reload" + 0.035*"like" + 0.023*"quick" + 0.019*"fast" + 0.014*"convenient" + 0.014*"liked" + 0.012*"get"
n
Topic: 4 nWords: 0.102*"great" + 0.051*"work" + 0.033*"ok" + 0.031*"worked" + 0.021*"time" + 0.021*"well" + 0.021*"br" + 0.019*"love" + 0.018*"process" + 0.016*"money"
n


In [60]:

for idx, topic in lda_model3.print_topics(-1):
    print("Topic: {} nWords: {}".format(idx, topic ))
    print("n")


Topic: 0 nWords: 0.074*"ok" + 0.044*"product" + 0.029*"great" + 0.025*"good" + 0.021*"wa" + 0.019*"really" + 0.018*"get" + 0.018*"like" + 0.017*"dollar" + 0.017*"would"
n
Topic: 1 nWords: 0.058*"card" + 0.041*"reload" + 0.038*"gift" + 0.035*"br" + 0.030*"get" + 0.029*"balance" + 0.021*"amazon" + 0.018*"wa" + 0.016*"credit" + 0.015*"account"
n
Topic: 2 nWords: 0.090*"card" + 0.066*"gift" + 0.052*"wa" + 0.036*"amazon" + 0.022*"use" + 0.020*"purchase" + 0.018*"balance" + 0.016*"money" + 0.014*"account" + 0.014*"time"
n
Topic: 3 nWords: 0.092*"easy" + 0.041*"time" + 0.039*"reload" + 0.032*"use" + 0.031*"find" + 0.030*"little" + 0.029*"hard" + 0.024*"first" + 0.021*"difficult" + 0.020*"bonus"
n
Topic: 4 nWords: 0.057*"good" + 0.049*"work" + 0.036*"use" + 0.026*"money" + 0.025*"know" + 0.023*"get" + 0.020*"pay" + 0.017*"would" + 0.017*"card" + 0.015*"expected"
n


In [61]:

for idx, topic in lda_model2.print_topics(-1):
    print("Topic: {} nWords: {}".format(idx, topic ))
    print("n")


Topic: 0 nWords: 0.076*"card" + 0.056*"gift" + 0.051*"amazon" + 0.034*"wa" + 0.027*"balance" + 0.027*"time" + 0.021*"use" + 0.018*"got" + 0.017*"payment" + 0.016*"could"
n
Topic: 1 nWords: 0.112*"card" + 0.053*"gift" + 0.035*"money" + 0.030*"credit" + 0.025*"amazon" + 0.020*"use" + 0.020*"wa" + 0.019*"balance" + 0.019*"time" + 0.018*"hour"
n
Topic: 2 nWords: 0.109*"wa" + 0.061*"card" + 0.045*"reload" + 0.042*"gift" + 0.028*"get" + 0.027*"bonus" + 0.023*"account" + 0.023*"never" + 0.018*"br" + 0.017*"balance"
n
Topic: 3 nWords: 0.053*"use" + 0.030*"great" + 0.026*"hard" + 0.024*"would" + 0.024*"ok" + 0.021*"time" + 0.020*"easy" + 0.019*"wa" + 0.019*"good" + 0.019*"card"
n
Topic: 4 nWords: 0.053*"br" + 0.040*"card" + 0.039*"amazon" + 0.024*"wa" + 0.022*"use" + 0.020*"reload" + 0.020*"know" + 0.020*"account" + 0.018*"would" + 0.017*"purchase"
n


In [62]:

for idx, topic in lda_model1.print_topics(-1):
    print("Topic: {} nWords: {}".format(idx, topic ))
    print("n")

Topic: 0 nWords: 0.042*"reload" + 0.029*"card" + 0.029*"never" + 0.027*"wa" + 0.021*"bonus" + 0.021*"br" + 0.020*"amazon" + 0.019*"gift" + 0.017*"would" + 0.017*"get"
n
Topic: 1 nWords: 0.037*"work" + 0.025*"account" + 0.022*"amazon" + 0.022*"wa" + 0.020*"card" + 0.014*"would" + 0.014*"time" + 0.014*"good" + 0.014*"gift" + 0.013*"bad"
n
Topic: 2 nWords: 0.045*"br" + 0.036*"wa" + 0.027*"use" + 0.019*"customer" + 0.018*"service" + 0.017*"card" + 0.017*"know" + 0.016*"amazon" + 0.014*"reload" + 0.014*"time"
n
Topic: 3 nWords: 0.096*"card" + 0.059*"gift" + 0.042*"amazon" + 0.037*"wa" + 0.021*"account" + 0.019*"credit" + 0.016*"get" + 0.016*"balance" + 0.015*"money" + 0.014*"use"
n
Topic: 4 nWords: 0.042*"get" + 0.034*"card" + 0.028*"wa" + 0.024*"reload" + 0.023*"money" + 0.016*"order" + 0.015*"amazon" + 0.015*"reloaded" + 0.012*"extra" + 0.011*"time"
n


In [72]:
df_cat1[(df_cat1['rating']==1)]

,rating,title,text,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,processed_text
51,1.0,"The added value was great, but no more",Purchasing an Amazon gift card came with an ad...,B00IX1I3G6,AEE2PYYDXJGV6ZO3YHZBBEMSUG4Q,1638164827641,0,True,"[purchasing, amazon, gift, card, came, added, ..."
234,1.0,Not happy,Not able you use all of my money,B00IX1I3G6,AHDDL5YTCQD64HSCZ3KXZQECH34Q,1539304079513,0,True,"[able, use, money]"
243,1.0,Issues,Tried to reload from American Express gift car...,B00IX1I3G6,AH3E7NHFIMIGMZ5K3F4RIB5ETSWA,1581626035802,0,True,"[tried, reload, american, express, gift, card,..."
726,1.0,Deceiving!!,Not what it seems to be. Didn’t give me my $10...,B00IX1I3G6,AESUDVNLFM4JHKTYTBVEPTPPEAWQ,1611762168735,0,True,"[seems, give, reload, award, return, something..."
958,1.0,Okay,Okay,B00IX1I3G6,AFQUKK77ZN2HMXLWLZW36T43QPHA,1549079760691,1,True,[okay]
...,...,...,...,...,...,...,...,...,...
152330,1.0,Understanding your procedures,I did not understand how the Reloadable gift w...,B00IX1I3G6,AHMGM5S2BLQAE4HCUMAH5NWI4WFA,1609886690607,0,True,"[understand, reloadable, gift, worked, thought..."
152337,1.0,One Star,I did not know I was 'loaded' let alone 'reloa...,B00IX1I3G6,AGBD7MPEGSTEZJLXAGP5X6TKSMPA,1442368697000,1,True,"[know, wa, let, alone, disputed, charge, amex]"
152346,1.0,Amazon Failed Me,Amazon makes it impossible to transfer funds f...,B00IX1I3G6,AFMNSBFGBCASBTZEH7EGCWPULK5Q,1493612569000,0,False,"[amazon, make, impossible, transfer, fund, ful..."
152379,1.0,gift card redemption,gift card says 100.00 but when I went to use i...,B00IX1I3G6,AHZWPVJ34GN3S4NKEXLPYXE2OQSQ,1562516491678,0,True,"[gift, card, say, went, use, said, redeemed, a..."
